_First code cell from custom template using [`jupyter_lab templates` extension](https://github.com/timkpaine/jupyterlab_templates)._

In [ ]:
# To get multiple outputs from one cell:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
from IPython.display import HTML, Markdown, Image
# for presentations:
#display(HTML("<style>.container { width:100% !important; }</style>"))

import sys
from pathlib import Path

def sys_info():
    frmt = '\nPython ver: {}\nPython env: {}\n'
    frmt += 'OS:         {}\nCurrent dir: {}\n'
    print(frmt.format(sys.version, 
                      Path(sys.prefix).name,
                      sys.platform,
                      Path.cwd()))


def add_to_sys_path(this_path, up=False):
    """
    Prepend this_path to sys.path.
    If up=True, path refers to parent folder (1 level up).
    """
    newp = Path(this_path).as_posix() # no str method (?)
    if up:
        newp = Path(this_path).parent.as_posix()

    msg = F'Path already in sys.path: {newp}'
    if newp not in sys.path:
        sys.path.insert(1, newp)
        msg = F'Path added to sys.path: {newp}'
    print(msg)

# if notebook inside another folder, eg ./notebooks:
nb_folder = 'notebooks'
add_to_sys_path(Path.cwd(), Path.cwd().name.startswith(nb_folder))


# Filtered dir() for method discovery:
def filter_dir(mdl, filter_str=None, start_with_str='_', exclude=True):
    """Filter dir(mdl) for method discovery.
       Input:
       :param mdl (object): module, optionally with submodule path(s), e.g. mdl.submdl1.submdl2.
       :param filter_str (str, None): filter all method names containing that string.
       :param start_with_str (str, '_'), exclude (bool, True): start_with_str and exclude work 
              together to perform search on non-dunder methods (default).
    """
    search_dir = [d for d in dir(mdl) if not d.startswith(start_with_str) == exclude]
    if filter_str is None:
        return search_dir
    else:
        filter_str = filter_str.lower()
        return [d for d in search_dir if d.lower().find(filter_str) != -1]


def get_project_dirs(which=['data', 'images'],
                     nb_folder='notebooks',
                     use_parent=True):
    '''Create folder(s) named in `which` at the parent level.'''
    dir_lst = []
    if Path.cwd().name.startswith(nb_folder) or use_parent:
        dir_fn = Path.cwd().parent.joinpath
    else:
        dir_fn = Path.cwd().joinpath
        
    for d in which:
        DIR = dir_fn(d)
        if not DIR.exists():
            Path.mkdir(DIR)
        dir_lst.append(DIR)
    return dir_lst

DIR_DATA, DIR_IMG = get_project_dirs()
    
import numpy as np
import scipy as sp
from scipy import stats as sps
import pandas as pd
#pd.set_option("display.max_colwidth", 200)

import matplotlib as mpl
from matplotlib import pyplot as plt
plt.ion()
plt.style.use('seaborn-muted')

from pprint import pprint as pp


def add_div(div_class='info', div_start='Tip:', 
            div_text='Some tip here', output_string=True):
    """
    Behaviour with default `output_string=True`:
    The cell is overwritten with the output, but the cell mode is still 'code',
    not 'markdown'.
    Workaround: After running the function, click on the new cell, press ESC, 
                type 'm', then run the new cell.
    If `output_string=False`, the output is displayed in an new cell with the 
    code cell visible.
    ```
    [x]
    add_div('alert-warning', 'Tip: ', 'some tip here', output_string=True)
    [x]
    <div class="alert alert-warning"><b>Tip: </b>some tip here</div>
    ```
    """
    accepted = ['info', 'warning', 'danger']
    div_class = div_class.lower()
    if div_class not in accepted:
        msg = f'<div class="alert"><b>Wrong class:&nbsp;</b> `div_start` not in: {accepted}.</div>'
        return Markdown(msg)
    
    div = f"""<div class="alert alert-{div_class}"><b>{div_start}&nbsp;&nbsp;</b>{div_text}</div>"""
    if output_string:
        return get_ipython().set_next_input(div, 'markdown')
    else:
        return Markdown(div)

    
def new_section(title='New section'):
    style = "text-align:center;background:#c2d3ef;padding:16px;color:#ffffff;font-size:2em;width:98%"
    div = f'<div style="{style}">{title}</div>'
    #return HTML('<div style="{}">{}</div>'.format(style, title))
    return get_ipython().set_next_input(div, 'markdown')


def show_versions():
    txt = '<pre><br>'
    txt += F'Python:\t\t{sys.version}<br>'
    txt += F'Python env:\t{Path(sys.prefix).name}<br>'
    txt += F'Numpy:\t\t{np.__version__}<br>'
    txt += F'Scipy:\t\t{sp.__version__}<br>'
    txt += F'Pandas:\t\t{pd.__version__}<br>'
    txt += F'Matplotlib:\t{mpl.__version__}<br>'
    txt += F'Currrent dir: {Path.cwd()}'
    txt += '</pre>'
    div = f"""<div class="alert alert-info"><b>Versions:</b><br>{txt}</div>"""
    return HTML(div)


# autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    get_ipython().run_line_magic('load_ext', 'autoreload')

%autoreload 2

#..................
sys_info()

no_wmark = False
try:
    %load_ext watermark
    %watermark
except ModuleNotFoundError:
    no_wmark = True

if no_wmark:
    show_versions()
else:
    %watermark -iv

---
---
# 'Bar chase': animated bar plot with covid-19 data

## Gif generation using matplotlib and pillow
---

## JHU CSSE covid-19 data 
## [times series](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series)  

```
time_series_covid19_confirmed_US.csv	Updates MI historical confirmed time series
time_series_covid19_deaths_US.csv	    Updates MI historical deaths time series

time_series_covid19_confirmed_global.csv	Updates global confirmed timeseries with MI mods
time_series_covid19_deaths_global.csv	    Correction for MI in 6/9 US deaths
time_series_covid19_recovered_global.csv	automated update
```
## Timeseries formats:
#### Global timeseries:
```
Province/State, Country/Region, Lat, Long, 1/22/20,...,6/17/20
```
=> aggregate by 'Country/Region'.  

#### US timeseries:
```
UID, iso2, iso3, code3, FIPS, Admin2, Province_State,Country_Region, Lat, Long_, Combined_Key, Population, 1/22/20,...,6/17/20
# Example:

     UID, iso2, iso3, code3,   FIPS, Admin2, Province_State, Country_Region,         Lat,       Long_,           Combined_Key, Population, 1/22/20,...,6/17/20
      16,   AS,  ASM,    16,   60.0,       , American Samoa,             US,     -14.271,     -170.132,   "American Samoa, US",     55641, 0, ..., 0
84001003,   US,  USA,   840, 1003.0, Baldwin,       Alabama,             US, 30.72774991, -87.72207058, "Baldwin, Alabama, US",    223234, 0, ..., 9
```
=> aggregate by 'Province_State'.

In [ ]:
# Local import
from barchase import dataset
from barchase import plotting
from barchase import utils

## Create the images output folder and subfolders

In [ ]:
# dict of png files for each dataset:
all_png_dirs = dataset.all_barchase_dirs()

# associated pictures folder:
global_conf_dir = all_png_dirs[('confirmed', 'global')]
utils.folder_info(global_conf_dir)

global_deaths_dir = all_png_dirs[('deaths', 'global')]
global_recov_dir = all_png_dirs[('recovered', 'global')]
us_conf_dir = all_png_dirs[('confirmed', 'US')]
us_deaths_dir = all_png_dirs[('deaths', 'US')]

## Data retrieval & processing: `dataset.get_JHU_CSSE_covid_ts()`
* The datasets are processed to output an additional column ('row_color') in order the keep the bars associated with the same color over the entire series

In [ ]:
global_ts=True
kind='deaths'
x_label='COVID-19 global deaths'

df, png_folder = dataset.get_JHU_CSSE_covid_ts(ts_kind=kind,
                                               global_ts=global_ts)

# new_data? 
if df is not None:
    #last col='row_color':
    days = df.columns.to_list()[:-1]
    #for naming gif:
    most_recent = days[-1].strftime('%Y_%m_%d')
    
    print(kind, F"global: {global_ts}", df.shape)
    df.head(1)

## Data plotting: `plotting.barh_topN()`

In [ ]:
# filter df by_country with [day, row_color]

d1 = days[-100]
d2 = days[-1]


plotting.barh_topN(d1, df=df, 
                   ts_kind=x_label,
                   #save_dir=png_folder,
                   save_as=None,  #i.e. don't save
                   show_fig=True)
        
plotting.barh_topN(d2, df=df, 
                   ts_kind=x_label,
                   save_as=None,
                   show_fig=True)

---
# All operations wrapped into `plotting.update_gif()`

## Gif generated from saved bar plots (png), then by using pil to open them & save to gif:
* Method allows saving of new images only.

# If plotting function has changed, it is recommended to redo all plots. 
## To do so, pass `replace_pics=True` to `update_gif()`. 

### Global deaths

In [ ]:
deaths_gif, df0 = plotting.update_gif(kind='deaths',
                                      global_ts=True,
                                      x_label='COVID-19 global deaths',
                                      replace_pics=True,
                                      plot_last_day=True)
deaths_gif

### US deaths

In [ ]:
us_deaths_gif, us0 = plotting.update_gif(kind='deaths',
                                         global_ts=False,
                                         x_label='COVID-19 deaths, US',
                                         per_multiple='thousands',
                                         replace_pics=True,
                                         plot_last_day=True)

### Global cases

In [ ]:
cases_gif, df1 = plotting.update_gif(kind='confirmed',
                                     global_ts=True,
                                     x_label='COVID-19 global cases',
                                     per_multiple='thousands',  #millions',
                                     replace_pics=True,
                                     plot_last_day=True)

### US cases

In [ ]:
us_cases_gif, us1 = plotting.update_gif(kind='confirmed',
                                        global_ts=False,
                                        x_label='COVID-19 cases, US',
                                        per_multiple='thousands',
                                        replace_pics=True,
                                        plot_last_day=True)